In [ ]:
import numpy as np
from tiatoolbox.wsicore.wsireader import VirtualWSIReader, WSIReader
import os
import matplotlib.pyplot as plt
import cv2
from utils import mm2_to_px, dist_to_px
import skimage
from scipy.spatial import Delaunay
from shapely.ops import polygonize, unary_union
import math
import shapely.geometry as geometry
import shapely
from PIL import Image, ImageDraw

In [ ]:
def calc_ratio(patch):
    ratio_patch = patch.copy()
    ratio_patch[ratio_patch > 1] = 1
    counts = np.unique(ratio_patch, return_counts=True)
    try:
        return (100 / counts[1][0]) * counts[1][1]
    except IndexError as ie:
        print(ie)
        print("Could not calculate ratio, using 0")
        return 0


def alpha_shape(points, alpha):
    def add_edge(edges, edge_points, coords, i, j):
        """
        Add a line between the i-th and j-th points,
        if not in the list already
        """
        if (i, j) in edges or (j, i) in edges:
            return
        edges.add((i, j))
        edge_points.append([coords[i], coords[j]])

    coords = [(i[0], i[1]) if type(i) or tuple else i for i in points]
    tri = Delaunay(coords)
    edges = set()
    edge_points = []

    # loop over triangles:
    # ia, ib, ic = indices of corner points of the
    # triangle
    for ia, ib, ic in tri.simplices:
        pa = coords[ia]
        pb = coords[ib]
        pc = coords[ic]
        # Lengths of sides of triangle
        a = math.sqrt((pa[0] - pb[0]) ** 2 + (pa[1] - pb[1]) ** 2)
        b = math.sqrt((pb[0] - pc[0]) ** 2 + (pb[1] - pc[1]) ** 2)
        c = math.sqrt((pc[0] - pa[0]) ** 2 + (pc[1] - pa[1]) ** 2)
        # Semiperimeter of triangle
        s = (a + b + c) / 2.0
        # Area of triangle by Heron's formula
        area = math.sqrt(s * (s - a) * (s - b) * (s - c))
        circum_r = a * b * c / (4.0 * area)
        # Here's the radius filter.
        # print circum_r
        if circum_r < 1 / alpha:
            add_edge(edges, edge_points, coords, ia, ib)
            add_edge(edges, edge_points, coords, ib, ic)
            add_edge(edges, edge_points, coords, ic, ia)
    m = geometry.MultiLineString(edge_points)
    triangles = list(polygonize(m))
    return unary_union(triangles), edge_points


def get_bulk(tumor_seg_mask):
    ratio = calc_ratio(tumor_seg_mask)
    print(ratio)
    mpp = 32
    min_size = 1.5

    if ratio > 50.0:
        kernel_diameter = dist_to_px(1000, mpp)
        min_size_px = mm2_to_px(min_size, mpp)
    else:
        min_size_px = mm2_to_px(1.0, mpp)
        kernel_diameter = dist_to_px(500, mpp)

    binary_tumor_mask = tumor_seg_mask.astype(bool)

    wsi_patch = skimage.morphology.remove_small_objects(
        binary_tumor_mask, min_size=mm2_to_px(0.005, mpp), connectivity=2
    )
    wsi_patch = wsi_patch.astype(np.uint8)

    kernel = cv2.getStructuringElement(
        cv2.MORPH_ELLIPSE, (kernel_diameter, kernel_diameter)
    )
    closing = cv2.morphologyEx(wsi_patch, cv2.MORPH_CLOSE, kernel)
    opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel)
    wsi_patch = opening
    wsi_patch = wsi_patch.astype(bool)
    wsi_patch_indexes = skimage.morphology.remove_small_objects(
        wsi_patch, min_size=min_size_px, connectivity=2
    )
    wsi_patch = wsi_patch.astype(np.uint8)
    wsi_patch[wsi_patch_indexes == False] = 0
    wsi_patch[wsi_patch_indexes == True] = 1

    points = np.argwhere(wsi_patch == 1)
    if len(points) == 0:
        print(f"no hull found")
        return wsi_patch

    alpha = 0.07
    concave_hull, _ = alpha_shape(points, alpha)
    if isinstance(concave_hull, shapely.geometry.polygon.Polygon) or isinstance(
        concave_hull, shapely.geometry.GeometryCollection
    ):
        polygons = [concave_hull]
    else:
        polygons = list(concave_hull.geoms)

    buffersize = dist_to_px(250, mpp)
    polygons = [
        geometry.Polygon(list(x.buffer(buffersize).exterior.coords)) for x in polygons
    ]

    coordinates = []
    for polygon in polygons:
        if polygon.area < min_size_px:
            continue

        coordinates.append(
            [(int(x[1]), int(x[0])) for x in polygon.boundary.coords[:-1]]
        )
    print(f"tumor bulk counts {len(coordinates)}")

    dimensions = tumor_seg_mask.shape
    img = Image.new("L", (dimensions[1], dimensions[0]), 0)
    for poly in coordinates:
        ImageDraw.Draw(img).polygon(poly, outline=1, fill=1)
    tumor_bulk = np.array(img, dtype=np.uint8)
    return tumor_bulk

In [ ]:
torch_seg_out_path = "output/seg_out"
tf_seg_out_path = "tf_seg_out/tempout"

wsi_dir = "/home/u1910100/Documents/Tiger_Data/wsitils/images"
eval_dir = "output/eval"

wsi_name = "104S.tif"
wsi_name_without_ext = os.path.splitext(wsi_name)[0]

reader = WSIReader.open(os.path.join(wsi_dir, wsi_name))
thumb = reader.slide_thumbnail(resolution=0.3125, units="power")

In [ ]:
tf_bulk_path = os.path.join(tf_seg_out_path, f"{wsi_name_without_ext}_bulk.npy")
tf_stroma_path = os.path.join(
    tf_seg_out_path, f"{wsi_name_without_ext}_bulk_stroma.npy"
)
tf_tumor_path = os.path.join(tf_seg_out_path, f"{wsi_name_without_ext}_bulk_tumor.npy")

tf_bulk_mask = np.load(tf_bulk_path)
tf_stroma_mask = np.load(tf_stroma_path)
tf_tumor_mask = np.load(tf_tumor_path)

fix, axes = plt.subplots(2, 2)

axes[0, 0].imshow(thumb)
axes[0, 0].set_title(wsi_name_without_ext)

axes[0, 1].imshow(tf_stroma_mask)
axes[0, 1].set_title("bulk stroma")

axes[1, 0].imshow(tf_tumor_mask)
axes[1, 0].set_title("bulk tumor")

axes[1, 1].imshow(tf_bulk_mask)
axes[1, 1].set_title("bulk")

for ax in axes.ravel():
    ax.axis("off")

plt.show()

In [ ]:
torch_seg_out_path = "output/seg_out"
tf_seg_out_path = "tf_seg_out/tempout"
wsi_dir = "/home/u1910100/Documents/Tiger_Data/wsitils/images"
eval_dir = "output/bulk_mask_v2"

total = 0
for image in os.listdir(wsi_dir):
    wsi_name_without_ext = os.path.splitext(image)[0]
    print(wsi_name_without_ext)

    reader = WSIReader.open(os.path.join(wsi_dir, image))
    thumb = reader.slide_thumbnail(resolution=0.3125, units="power")

    try:
        stroma_mask = np.load(
            os.path.join(torch_seg_out_path, f"{wsi_name_without_ext}_stroma.npy")
        )
        tumor_mask = np.load(
            os.path.join(torch_seg_out_path, f"{wsi_name_without_ext}_tumor.npy")
        )
    except:
        continue

    ratio = calc_ratio(tumor_mask)
    if ratio < 0.1:
        kernel_diameter = dist_to_px(500, 32)
        kernel = cv2.getStructuringElement(
            cv2.MORPH_ELLIPSE, (kernel_diameter, kernel_diameter)
        )
        tumor_mask = cv2.dilate(tumor_mask, kernel, iterations=1)

    bulk_mask = get_bulk(tumor_mask)

    if np.count_nonzero(bulk_mask) > 0:
        tumor_bulk = tumor_mask * bulk_mask
        stroma_bulk = stroma_mask * bulk_mask
    else:
        tumor_bulk = tumor_mask
        stroma_bulk = stroma_mask

    np.save(
        os.path.join(torch_seg_out_path, f"{wsi_name_without_ext}_tumor_bulk.npy"),
        tumor_bulk,
    )
    np.save(
        os.path.join(torch_seg_out_path, f"{wsi_name_without_ext}_stroma_bulk.npy"),
        stroma_bulk,
    )
    np.save(
        os.path.join(torch_seg_out_path, f"{wsi_name_without_ext}_bulk.npy"), bulk_mask
    )

    fig, axes = plt.subplots(2, 3)
    fig.tight_layout()
    axes[0, 0].imshow(thumb)
    axes[0, 0].set_title(wsi_name_without_ext)

    axes[0, 1].imshow(stroma_mask)
    axes[0, 1].set_title("stroma")

    axes[0, 2].imshow(tumor_mask)
    axes[0, 2].set_title("tumor")

    axes[1, 0].imshow(bulk_mask)
    axes[1, 0].set_title("bulk")

    axes[1, 1].imshow(tumor_bulk)
    axes[1, 1].set_title("tumor_bulk")

    axes[1, 2].imshow(stroma_bulk)
    axes[1, 2].set_title("stroma_bulk")

    for ax in axes.ravel():
        ax.axis("off")
    plt.show()
    total += 1
    # save_path = os.path.join(eval_dir, f"{wsi_name_without_ext}.png")
    # fig.savefig(
    #     save_path,
    #     bbox_inches="tight",
    #     pad_inches=0,
    # )
    # fig.clf()
    plt.close()
print(total)

In [ ]:
tumor_mask = np.load(
    os.path.join(torch_seg_out_path, f"TC_S01_P000041_C0001_B101_tumor.npy")
)
print(np.count_nonzero(tumor_mask))
plt.imshow(tumor_mask)
plt.show()
print(calc_ratio(tumor_mask))
kernel_diameter = dist_to_px(400, 32)
kernel = cv2.getStructuringElement(
    cv2.MORPH_ELLIPSE, (kernel_diameter, kernel_diameter)
)
img_dilation = cv2.dilate(tumor_mask, kernel, iterations=1)
print(np.count_nonzero(img_dilation))
plt.imshow(img_dilation)
plt.show()

bulk_mask = get_bulk(img_dilation)
plt.imshow(bulk_mask)
plt.show()